In [1]:
import time

from utils import dataStructures, dataset, parallelizer
from utils.generators import markovian
import numpy as np


In [2]:
def print_correct(mela_id='22', janya_id='22_a', num_surr=100, surr_serial=False, time_cal=None):
    dat = dataset.GetRagaSongCoordsConcat(mela_id, janya_id)
    # dat1 = dataset.GetRagaSongCoords(janya_id)
    if time_cal is not None:
        assert isinstance(time_cal, dict)
    print("Starting surrogate generation...")
    t1 = time.perf_counter()
    d_surr1 = markovian.GenerateForRaga(mela_id, num_surr, serial=surr_serial)
    d_surr2 = markovian.GenerateForRaga(janya_id, num_surr, serial=surr_serial)
    t2 = time.perf_counter()
    for key in dat:
        dat[key] = dataStructures.PackTuples(*dat[key])

    # for key in dat1:
    #     dat1[key] = dataStructures.PackTuples(*dat1[key])

    d_fin1 = dataset.ConcatenateDicts(dat, d_surr1, d_surr2)
    adj = dataset.GetAdjustedMelodies(d_fin1, pre_min_len=43680)
    print(f"adjusted Melody length: {len(list(adj.values())[0])}")
    print("finding causality....")
    t3 = time.perf_counter()
    tc, totc, strengths = parallelizer.TrueLZPCausality_listParallel(list(adj.values()), list(adj.keys()), mela=dataset.GetRagaFromRagaId(mela_id))
    t4 = time.perf_counter()

    if time_cal is not None:
        time_cal["Surrogate{}".format(mela_id)] = t2 - t1
        time_cal["Causality{}".format(mela_id)] = t4 - t3
    return tc, totc, strengths

def get_generation_time50(mela_id='22', janya_id='22_a', surr_serial=True):
    dat = dataset.GetRagaSongCoordsConcat(mela_id)
    dat1 = dataset.GetRagaSongCoords(janya_id)
    print("Starting surrogate generation...")
    t1 = time.perf_counter()
    d_surr1 = markovian.GenerateForRaga(mela_id, 50, serial=surr_serial)
    d_surr2 = markovian.GenerateForRaga(janya_id, 50, serial=surr_serial)
    t2 = time.perf_counter()
    for key in dat:
        dat[key] = dataStructures.PackTuples(*dat[key])

    for key in dat1:
        dat1[key] = dataStructures.PackTuples(*dat1[key])

    d_fin1 = dataset.ConcatenateDicts(dat, d_surr1, dat1, d_surr2)
    adj = dataset.GetAdjustedMelodies(d_fin1)

    return t2 - t1


def ragaIterator(n_iters, mela, janya, num_surrogates=100, write=True, times_it=None):
    if times_it is not None:
        assert isinstance(times_it, dict)
        times_it["Surrogate{}".format(mela)] = []
        times_it["Causality{}".format(mela)] = []
    tcs = []
    strengths = []
    totcs = []
    f = open("/home/efm-workstation/surr_stats/{}/truecauses{}.txt".format(mela, num_surrogates), "a")
    t1 = time.perf_counter()
    for i in range(n_iters):
        print("iteration: ", i + 1)
        tm = {}
        if i % 2 != 0:
            tc, totc, st = print_correct(mela, janya, num_surr=num_surrogates, surr_serial=True, time_cal=tm)
        else:
            tc, totc, st = print_correct(mela, janya, num_surr=num_surrogates, surr_serial=True, time_cal=tm)
        if times_it is not None:
            times_it["Surrogate{}".format(mela)].append(tm["Surrogate{}".format(mela)])
            times_it["Causality{}".format(mela)].append(tm["Causality{}".format(mela)])
        if write:
            f.write("iteration: {}/{}, {}/{}; time taken: {}/{}\n".format(i+1, n_iters, tc, totc,
                                                                          tm["Surrogate{}".format(mela)],
                                                                          tm["Causality{}".format(mela)]))
        t = time.perf_counter()
        print("time elapsed: ", (t - t1))
        t1 = t
        tcs.append(tc)
        totcs.append(totc)
        strengths.append(st)
    f.close()
    return tcs, totcs, strengths

avg_conns = lambda t_c : np.array(t_c).mean()

## 100 Surrogates

In [3]:
t_dict = {}

In [41]:
tc65, totc65, ss65 = ragaIterator(10, '65', '29_h', num_surrogates=100, times_it=t_dict)

iteration:  1
Starting surrogate generation...


100%|██████████| 100/100 [00:29<00:00,  3.43it/s]


adjusted Melody length: 43680
finding causality....
6213 true causes out of 11024 total connections
time elapsed:  758.5425840989919
iteration:  2
Starting surrogate generation...


100%|██████████| 100/100 [00:23<00:00,  4.33it/s]


adjusted Melody length: 43680
finding causality....
6106 true causes out of 11024 total connections
time elapsed:  770.2031381230336
iteration:  3
Starting surrogate generation...


...

adjusted Melody length: 43680
finding causality....
6374 true causes out of 11024 total connections
time elapsed:  764.4010294040199
iteration:  9
Starting surrogate generation...


100%|██████████| 100/100 [00:32<00:00,  3.12it/s]


adjusted Melody length: 43680
finding causality....
5714 true causes out of 11024 total connections
time elapsed:  778.6158331959741
iteration:  10
Starting surrogate generation...


100%|██████████| 100/100 [00:51<00:00,  1.95it/s]


adjusted Melody length: 43680
finding causality....
5857 true causes out of 11024 total connections
time elapsed:  766.1013949390035


In [42]:
avg_conns(tc65), avg_conns(t_dict["Surrogate65"]), avg_conns(t_dict["Causality65"])

(6021.1, 78.61135326190852, 667.6914366678917)

In [43]:
tc15, totc15, ss15 = ragaIterator(10, '15', '15_m', num_surrogates=100, times_it=t_dict)

iteration:  1
Starting surrogate generation...


100%|██████████| 100/100 [00:38<00:00,  2.58it/s]


adjusted Melody length: 43680
finding causality....
8929 true causes out of 11024 total connections
time elapsed:  709.9851495609619
iteration:  2
Starting surrogate generation...


100%|██████████| 100/100 [00:25<00:00,  3.94it/s]


adjusted Melody length: 43680
finding causality....
8679 true causes out of 11024 total connections
time elapsed:  702.5166410560487
iteration:  3
Starting surrogate generation...


...

adjusted Melody length: 43680
finding causality....
8542 true causes out of 11024 total connections
time elapsed:  713.063716355944
iteration:  9
Starting surrogate generation...


100%|██████████| 100/100 [00:44<00:00,  2.26it/s]


adjusted Melody length: 43680
finding causality....
8802 true causes out of 11024 total connections
time elapsed:  714.2705530250678
iteration:  10
Starting surrogate generation...


100%|██████████| 100/100 [00:28<00:00,  3.54it/s]


adjusted Melody length: 43680
finding causality....
9008 true causes out of 11024 total connections
time elapsed:  697.0699470799882


In [44]:
avg_conns(tc15), avg_conns(t_dict["Surrogate15"]), avg_conns(t_dict["Causality15"])

(8735.2, 71.56543712898856, 621.608187388)

In [45]:
tc8, totc8, ss8 = ragaIterator(10, '8', '8_d', num_surrogates=100, times_it=t_dict)

iteration:  1
Starting surrogate generation...


100%|██████████| 100/100 [00:58<00:00,  1.71it/s]


adjusted Melody length: 43680
finding causality....
1482 true causes out of 11448 total connections
time elapsed:  900.8957279680762
iteration:  2
Starting surrogate generation...


100%|██████████| 100/100 [00:36<00:00,  2.70it/s]


adjusted Melody length: 43680
finding causality....
2233 true causes out of 11448 total connections
time elapsed:  904.8804481339175
iteration:  3
Starting surrogate generation...


...

adjusted Melody length: 43680
finding causality....
2263 true causes out of 11448 total connections
time elapsed:  902.1084413579665
iteration:  9
Starting surrogate generation...


100%|██████████| 100/100 [00:31<00:00,  3.18it/s]


adjusted Melody length: 43680
finding causality....
2076 true causes out of 11448 total connections
time elapsed:  888.8213430270553
iteration:  10
Starting surrogate generation...


100%|██████████| 100/100 [00:24<00:00,  4.02it/s]


adjusted Melody length: 43680
finding causality....
2039 true causes out of 11448 total connections
time elapsed:  911.6524298959412


In [46]:
avg_conns(tc8), avg_conns(t_dict["Surrogate8"]), avg_conns(t_dict["Causality8"])

(1877.8, 81.56304458830273, 808.4431912748842)

In [4]:
tc22, totc22, ss22 = ragaIterator(10, '22', '22_a', num_surrogates=100, times_it=t_dict)

iteration:  1
Starting surrogate generation...


100%|██████████| 100/100 [00:40<00:00,  2.49it/s]


adjusted Melody length: 43680
finding causality....
7777 true causes out of 10918 total connections
time elapsed:  775.1191736069999
iteration:  2
Starting surrogate generation...


100%|██████████| 100/100 [00:33<00:00,  3.02it/s]


adjusted Melody length: 43680
finding causality....
7681 true causes out of 10918 total connections
time elapsed:  751.3098408629999
iteration:  3
Starting surrogate generation...


...

adjusted Melody length: 43680
finding causality....
7134 true causes out of 10918 total connections
time elapsed:  769.3840020909993
iteration:  9
Starting surrogate generation...


100%|██████████| 100/100 [00:46<00:00,  2.15it/s]


adjusted Melody length: 43680
finding causality....
6914 true causes out of 10918 total connections
time elapsed:  776.2038353220005
iteration:  10
Starting surrogate generation...


100%|██████████| 100/100 [00:46<00:00,  2.17it/s]


adjusted Melody length: 43680
finding causality....
6685 true causes out of 10918 total connections
time elapsed:  765.856024617


In [5]:
avg_conns(tc22), avg_conns(t_dict["Surrogate22"]), avg_conns(t_dict["Causality22"])

(7202.5, 74.12116545719991, 673.4122535278999)

In [6]:
tc28, totc28, ss28 = ragaIterator(10, '28', '28_k', num_surrogates=100, times_it=t_dict)

iteration:  1
Starting surrogate generation...


100%|██████████| 100/100 [00:53<00:00,  1.88it/s]


adjusted Melody length: 43680
finding causality....
5201 true causes out of 10920 total connections
time elapsed:  901.3206878130004
iteration:  2
Starting surrogate generation...


100%|██████████| 100/100 [01:01<00:00,  1.63it/s]


adjusted Melody length: 43680
finding causality....
5318 true causes out of 10920 total connections
time elapsed:  907.9305880369993
iteration:  3
Starting surrogate generation...


...

adjusted Melody length: 43680
finding causality....
5340 true causes out of 10920 total connections
time elapsed:  874.3105226890002
iteration:  9
Starting surrogate generation...


100%|██████████| 100/100 [00:48<00:00,  2.07it/s]


adjusted Melody length: 43680
finding causality....
5343 true causes out of 10920 total connections
time elapsed:  883.6631521609997
iteration:  10
Starting surrogate generation...


100%|██████████| 100/100 [00:37<00:00,  2.67it/s]


adjusted Melody length: 43680
finding causality....
5858 true causes out of 10920 total connections
time elapsed:  873.109005657001


In [7]:
avg_conns(tc28), avg_conns(t_dict["Surrogate28"]), avg_conns(t_dict["Causality28"])

(5443.7, 87.33458145420009, 780.9243078063)

In [8]:
tc29, totc29, ss29 = ragaIterator(10, '29', '29_h', num_surrogates=100, times_it=t_dict)

iteration:  1
Starting surrogate generation...


100%|██████████| 100/100 [00:27<00:00,  3.62it/s]


adjusted Melody length: 43680
finding causality....
8096 true causes out of 11024 total connections
time elapsed:  811.1475962440018
iteration:  2
Starting surrogate generation...


100%|██████████| 100/100 [00:41<00:00,  2.44it/s]


adjusted Melody length: 43680
finding causality....
8483 true causes out of 11024 total connections
time elapsed:  822.283582865999
iteration:  3
Starting surrogate generation...


...

adjusted Melody length: 43680
finding causality....
8579 true causes out of 11024 total connections
time elapsed:  807.5053986649982
iteration:  9
Starting surrogate generation...


100%|██████████| 100/100 [00:32<00:00,  3.08it/s]


adjusted Melody length: 43680
finding causality....
7932 true causes out of 11024 total connections
time elapsed:  819.2318969290027
iteration:  10
Starting surrogate generation...


100%|██████████| 100/100 [00:31<00:00,  3.16it/s]


adjusted Melody length: 43680
finding causality....
8427 true causes out of 11024 total connections
time elapsed:  804.8484866289982


In [9]:
avg_conns(tc29), avg_conns(t_dict["Surrogate29"]), avg_conns(t_dict["Causality29"])

(8454.7, 70.77880628050043, 727.6982087450003)

In [14]:
t_dict


{'Surrogate15': [53.26868949300001,
  57.416984490000004,
  65.63219295199997,
  74.16984343900003,
  74.61059070900001,
  58.23887661700019,
  68.6738725109999,
  48.17700101700029,
  55.440457104000416,
  74.29063077299998],
 'Causality15': [592.3598863940001,
  591.381109908,
  592.825908996,
  587.7917414099998,
  595.133812782,
  587.7548258839997,
  594.1940579100001,
  588.1832207909993,
  593.7933637599999,
  595.1086932379994],
 'Surrogate8': [89.54098272700048,
  58.45198296499984,
  73.39187069900072,
  68.46543324499908,
  54.848082572998464,
  73.66247698000006,
  97.71908451700074,
  57.23644118000084,
  78.49121733200082,
  67.34753957300018],
 'Causality8': [755.0378918679999,
  759.8611159019993,
  758.2308762759985,
  752.4552812790007,
  760.3299102830006,
  759.4336225900006,
  759.0644153419998,
  769.2846034569993,
  761.8674483529994,
  759.9956650980002],
 'Surrogate22': [68.32396897400031,
  58.05357082699993,
  62.23361633500099,
  91.03038601200024,
  61.1071

## 50 Surrogates

In [10]:
t_dict = {}

In [11]:
tc65, totc65, ss65 = ragaIterator(10, '65', '29_h', num_surrogates=50, times_it=t_dict)

iteration:  1
Starting surrogate generation...


100%|██████████| 50/50 [00:12<00:00,  3.96it/s]


adjusted Melody length: 43680
finding causality....
1615 true causes out of 3024 total connections
time elapsed:  218.37166827799956
iteration:  2
Starting surrogate generation...


100%|██████████| 50/50 [00:21<00:00,  2.38it/s]


adjusted Melody length: 43680
finding causality....
1669 true causes out of 3024 total connections
time elapsed:  240.33316805800132
iteration:  3
Starting surrogate generation...


...

adjusted Melody length: 43680
finding causality....
1747 true causes out of 3024 total connections
time elapsed:  212.7220547470024
iteration:  9
Starting surrogate generation...


100%|██████████| 50/50 [00:20<00:00,  2.41it/s]


adjusted Melody length: 43680
finding causality....
1608 true causes out of 3024 total connections
time elapsed:  233.68813059999957
iteration:  10
Starting surrogate generation...


100%|██████████| 50/50 [00:14<00:00,  3.53it/s]


adjusted Melody length: 43680
finding causality....
1480 true causes out of 3024 total connections
time elapsed:  219.93920594000156


In [12]:
avg_conns(tc65), avg_conns(t_dict["Surrogate65"]), avg_conns(t_dict["Causality65"])

(1644.6, 37.424173020300074, 182.3727227753996)

In [13]:
tc15, totc15, ss15 = ragaIterator(10, '15', '15_m', num_surrogates=50, times_it=t_dict)

iteration:  1
Starting surrogate generation...


100%|██████████| 50/50 [00:16<00:00,  2.97it/s]


adjusted Melody length: 43680
finding causality....
2369 true causes out of 3024 total connections
time elapsed:  212.382418606001
iteration:  2
Starting surrogate generation...


100%|██████████| 50/50 [00:16<00:00,  3.11it/s]


adjusted Melody length: 43680
finding causality....
2434 true causes out of 3024 total connections
time elapsed:  204.5261949679989
iteration:  3
Starting surrogate generation...


...

adjusted Melody length: 43680
finding causality....
2391 true causes out of 3024 total connections
time elapsed:  200.65594127000077
iteration:  9
Starting surrogate generation...


100%|██████████| 50/50 [00:24<00:00,  2.01it/s]


adjusted Melody length: 43680
finding causality....
2457 true causes out of 3024 total connections
time elapsed:  214.36199954900076
iteration:  10
Starting surrogate generation...


100%|██████████| 50/50 [00:21<00:00,  2.33it/s]


adjusted Melody length: 43680
finding causality....
2349 true causes out of 3024 total connections
time elapsed:  217.20283365499927


In [14]:
avg_conns(tc15), avg_conns(t_dict["Surrogate15"]), avg_conns(t_dict["Causality15"])

(2342.1, 35.84443851620017, 169.2937976976995)

In [15]:
tc8, totc8, ss8 = ragaIterator(10, '8', '8_d', num_surrogates=50, times_it=t_dict)

iteration:  1
Starting surrogate generation...


100%|██████████| 50/50 [00:16<00:00,  2.99it/s]


adjusted Melody length: 43680
finding causality....
619 true causes out of 3248 total connections
time elapsed:  270.9603171999988
iteration:  2
Starting surrogate generation...


100%|██████████| 50/50 [00:18<00:00,  2.70it/s]


adjusted Melody length: 43680
finding causality....
782 true causes out of 3248 total connections
time elapsed:  278.37013650600056
iteration:  3
Starting surrogate generation...


...

adjusted Melody length: 43680
finding causality....
612 true causes out of 3248 total connections
time elapsed:  276.27639988900046
iteration:  9
Starting surrogate generation...


100%|██████████| 50/50 [00:17<00:00,  2.89it/s]


adjusted Melody length: 43680
finding causality....
553 true causes out of 3248 total connections
time elapsed:  259.0935400140006
iteration:  10
Starting surrogate generation...


100%|██████████| 50/50 [00:30<00:00,  1.63it/s]


adjusted Melody length: 43680
finding causality....
703 true causes out of 3248 total connections
time elapsed:  288.6824951169983


In [16]:
avg_conns(tc8), avg_conns(t_dict["Surrogate8"]), avg_conns(t_dict["Causality8"])

(668.6, 40.217083348899905, 223.51942214590053)

In [17]:
tc22, totc22, ss22 = ragaIterator(10, '22', '22_a', num_surrogates=50, times_it=t_dict)

iteration:  1
Starting surrogate generation...


100%|██████████| 50/50 [00:18<00:00,  2.70it/s]


adjusted Melody length: 43680
finding causality....
1883 true causes out of 2968 total connections
time elapsed:  226.83664754599886
iteration:  2
Starting surrogate generation...


100%|██████████| 50/50 [00:14<00:00,  3.39it/s]


adjusted Melody length: 43680
finding causality....
2022 true causes out of 2968 total connections
time elapsed:  230.5618512630026
iteration:  3
Starting surrogate generation...


...

adjusted Melody length: 43680
finding causality....
2053 true causes out of 2968 total connections
time elapsed:  221.2663969109999
iteration:  9
Starting surrogate generation...


100%|██████████| 50/50 [00:12<00:00,  3.89it/s]


adjusted Melody length: 43680
finding causality....
1665 true causes out of 2968 total connections
time elapsed:  225.04745852699853
iteration:  10
Starting surrogate generation...


100%|██████████| 50/50 [00:12<00:00,  4.10it/s]


adjusted Melody length: 43680
finding causality....
1835 true causes out of 2968 total connections
time elapsed:  214.0779628510063


In [18]:
avg_conns(tc22), avg_conns(t_dict["Surrogate22"]), avg_conns(t_dict["Causality22"])

(1899.4, 33.20455161340105, 182.73843879880042)

In [19]:
tc28, totc28, ss28 = ragaIterator(10, '28', '28_k', num_surrogates=50, times_it=t_dict)

iteration:  1
Starting surrogate generation...


100%|██████████| 50/50 [00:19<00:00,  2.58it/s]


adjusted Melody length: 43680
finding causality....
1583 true causes out of 2970 total connections
time elapsed:  246.19132544999593
iteration:  2
Starting surrogate generation...


100%|██████████| 50/50 [00:24<00:00,  2.02it/s]


adjusted Melody length: 43680
finding causality....
1491 true causes out of 2970 total connections
time elapsed:  265.5274024860046
iteration:  3
Starting surrogate generation...


...

adjusted Melody length: 43680
finding causality....
1428 true causes out of 2970 total connections
time elapsed:  252.37233264899987
iteration:  9
Starting surrogate generation...


100%|██████████| 50/50 [00:18<00:00,  2.77it/s]


adjusted Melody length: 43680
finding causality....
1428 true causes out of 2970 total connections
time elapsed:  248.53994721799972
iteration:  10
Starting surrogate generation...


100%|██████████| 50/50 [00:20<00:00,  2.43it/s]


adjusted Melody length: 43680
finding causality....
1588 true causes out of 2970 total connections
time elapsed:  254.24143342200114


In [20]:
avg_conns(tc28), avg_conns(t_dict["Surrogate28"]), avg_conns(t_dict["Causality28"])

(1493.6, 39.67997062440045, 208.65607820050136)

In [21]:
tc29, totc29, ss29 = ragaIterator(10, '29', '29_h', num_surrogates=50, times_it=t_dict)

iteration:  1
Starting surrogate generation...


100%|██████████| 50/50 [00:17<00:00,  2.93it/s]


adjusted Melody length: 43680
finding causality....
2319 true causes out of 3024 total connections
time elapsed:  243.95732997899904
iteration:  2
Starting surrogate generation...


100%|██████████| 50/50 [00:14<00:00,  3.35it/s]


adjusted Melody length: 43680
finding causality....
2328 true causes out of 3024 total connections
time elapsed:  246.67933081799856
iteration:  3
Starting surrogate generation...


...

adjusted Melody length: 43680
finding causality....
2327 true causes out of 3024 total connections
time elapsed:  240.7403798680025
iteration:  9
Starting surrogate generation...


100%|██████████| 50/50 [00:16<00:00,  3.08it/s]


adjusted Melody length: 43680
finding causality....
2405 true causes out of 3024 total connections
time elapsed:  235.35074236599758
iteration:  10
Starting surrogate generation...


100%|██████████| 50/50 [00:25<00:00,  1.96it/s]


adjusted Melody length: 43680
finding causality....
2141 true causes out of 3024 total connections
time elapsed:  245.82174390100408


In [22]:
avg_conns(tc29), avg_conns(t_dict["Surrogate29"]), avg_conns(t_dict["Causality29"])

(2322.1, 37.971970005199545, 197.75677085439892)

In [ ]:
t_dict

In [3]:
gentimes8 = []

for _ in range(10):
    gentimes8.append(get_generation_time50('8', '8_d'))
    gentimes8.append(get_generation_time50('15', '15_m'))
    gentimes8.append(get_generation_time50('22', '22_a'))
    gentimes8.append(get_generation_time50('28', '28_k'))
    gentimes8.append(get_generation_time50('29', '29_h'))

Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting s

In [5]:
gentime8 = np.array(gentimes8[0:10]).mean()
gentime15 = np.array(gentimes8[10:20]).mean()
gentime22 = np.array(gentimes8[20:30]).mean()
gentime28 = np.array(gentimes8[30:40]).mean()
gentime29 = np.array(gentimes8[40:50]).mean()

gentime8, gentime15, gentime22, gentime28, gentime29

(33.169615245600006,
 32.59101138219999,
 32.932949019099965,
 31.621566379899967,
 32.94254417129996)

## 150 surrogates

In [6]:
t_dict = {}

In [ ]:
tc15, totc15, ss15 = ragaIterator(10, '15', '15_m', num_surrogates=150, times_it=t_dict)

iteration:  1
Starting surrogate generation...


100%|██████████| 150/150 [00:54<00:00,  2.74it/s]


finding causality....
20336 true causes out of 24024 total connections
time elapsed:  1912.7792934960016
iteration:  2
Starting surrogate generation...


100%|██████████| 150/150 [00:35<00:00,  4.24it/s]


finding causality....
19595 true causes out of 24024 total connections
time elapsed:  1888.3809031659985
iteration:  3
Starting surrogate generation...


...

finding causality....
20604 true causes out of 24024 total connections
time elapsed:  1900.1705254010012
iteration:  5
Starting surrogate generation...


100%|██████████| 150/150 [00:51<00:00,  2.91it/s]


finding causality....
20164 true causes out of 24024 total connections
time elapsed:  1918.1216722840036
iteration:  6
Starting surrogate generation...


100%|██████████| 150/150 [00:45<00:00,  3.26it/s]


finding causality....


In [ ]:
avg_conns(tc15), avg_conns(t_dict["Surrogate15"]), avg_conns(t_dict["Causality15"])

In [12]:
tc8, totc8, ss8 = ragaIterator(10, '8', '8_d', num_surrogates=150, times_it=t_dict)

iteration:  1
Starting surrogate generation...
finding causality....
14463 true causes out of 24648 total connections
time elapsed:  2377.1097622410016
iteration:  2
Starting surrogate generation...
finding causality....
15204 true causes out of 24648 total connections
time elapsed:  2329.3596639659954
iteration:  3
Starting surrogate generation...
finding causality....
15244 true causes out of 24648 total connections
time elapsed:  2377.415402065002
iteration:  4
Starting surrogate generation...
finding causality....


Process ForkPoolWorker-1040:
Process ForkPoolWorker-1047:
Process ForkPoolWorker-1052:
Process ForkPoolWorker-1055:
Process ForkPoolWorker-1034:
Process ForkPoolWorker-1054:
Process ForkPoolWorker-1051:
Process ForkPoolWorker-1045:
Process ForkPoolWorker-1046:
Process ForkPoolWorker-1049:
Process ForkPoolWorker-1039:
Process ForkPoolWorker-1041:
Process ForkPoolWorker-1056:
Process ForkPoolWorker-1035:
Process ForkPoolWorker-1033:
Process ForkPoolWorker-1043:
Process ForkPoolWorker-1048:
Process ForkPoolWorker-1053:
Process ForkPoolWorker-1038:
Process ForkPoolWorker-1050:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/abhisheknandekar/miniconda3/envs/raga/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/abhisheknandekar/miniconda3

KeyboardInterrupt: 

In [ ]:
avg_conns(tc8), avg_conns(t_dict["Surrogate8"]), avg_conns(t_dict["Causality8"])

In [ ]:
tc22, totc22, ss22 = ragaIterator(10, '22', '22_a', num_surrogates=150, times_it=t_dict)

In [ ]:
avg_conns(tc22), avg_conns(t_dict["Surrogate22"]), avg_conns(t_dict["Causality22"])

In [ ]:
tc28, totc28, ss28 = ragaIterator(10, '28', '28_k', num_surrogates=150, times_it=t_dict)

In [ ]:
avg_conns(tc28), avg_conns(t_dict["Surrogate28"]), avg_conns(t_dict["Causality28"])

In [ ]:
tc29, totc29, ss29 = ragaIterator(10, '29', '29_h', num_surrogates=150, times_it=t_dict)

In [ ]:
avg_conns(tc29), avg_conns(t_dict["Surrogate29"]), avg_conns(t_dict["Causality29"])

In [ ]:
t_dict